In [1]:
import sys
sys.path.append('..')

### Set OpenAI key 

In [2]:
import os
import configparser

config = configparser.ConfigParser()
config.read('../../.secrets.ini')
openai_api_key = config['OPENAI']['OPENAI_API_KEY']

os.environ.update({'OPENAI_API_KEY': openai_api_key})

### chain 설정

In [3]:
from langchain.prompts import load_prompt, PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback
from langchain.embeddings import OpenAIEmbeddings

In [4]:
template = '''
I want you to be the writer who creates the table of contents for the article.
I will give you the topic of the article, and I will need you to generate the table of contents.
Only say the table of contents. But if there is none, say nothing, and don't give me an explanation.
Answer in Korean.

topic 1: """디지털화 시대의 화폐와 경제: 암호화폐의 등장과 그 영향"""
table of contents 1: 
1. 서론
- 디지털화 시대와 경제의 관계
- 암호화폐에 대한 간략한 소개
2. 암호화폐의 역사
- 비트코인의 출현: 블록체인과 디센트럴라이제이션
- 주요 암호화폐들의 발전과 다양성 
3. 암호화폐의 경제적 영향
- 통화정책에 미치는 영향: 중앙은행과의 관계
- 국제거래에서의 암호화폐 활용
- 경제 내에서의 암호화폐로 인한 기회와 위험
4. 실제 사례 연구
- 암호화폐를 사용하는 기업들의 사례
- 국가 수준에서 암호화폐를 인정하거나 규제하는 사례
5. 미래 전망
- 암호화폐와 전통 화폐의 관계
- 디지털화 시대에서의 암호화폐의 장기적인 영향
6. 결론
- 디지털화 시대의 경제 트렌드와 암호화폐의 역할
- 글쓰기 주제와 관련된 개인적인 견해나 생각

topic 2: """{purpose}"""
table of contents 2:
'''

In [5]:
class TableChain:
    def __init__(self, table_template=None, table_template_path='../openai_skt/models/templates/table_prompt.json', verbose=False) -> None:
        if table_template is not None:
            self.table_prompt = PromptTemplate(
                    input_variables=["purpose"],
                    template=table_template,
                )
        else:
            self.table_template_path = table_template_path
            self.table_prompt = load_prompt(self.table_template_path)
        self.llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
        self.verbose = verbose
        self.table_chain = LLMChain(llm=self.llm, prompt=self.table_prompt, verbose=self.verbose)

    def run(self, purpose:str=None):
        input_dict = {'purpose': purpose}
        table = self.table_chain.run(input_dict)
        return table

    async def arun(self, purpose:str=None):
        input_dict = {'purpose': purpose}
        table = await self.table_chain.arun(input_dict)
        return table

In [6]:
table_chain = TableChain(table_template=template, verbose=True)

In [7]:
result = table_chain.run(purpose="축구가 세계적으로 인기있는 스포츠인 이유")



> Entering new LLMChain chain...
Prompt after formatting:

I want you to be the writer who creates the table of contents for the article.
I will give you the topic of the article, and I will need you to generate the table of contents.
Only say the table of contents. But if there is none, say nothing, and don't give me an explanation.
Answer in Korean.

topic 1: """디지털화 시대의 화폐와 경제: 암호화폐의 등장과 그 영향"""
table of contents 1: 
1. 서론
- 디지털화 시대와 경제의 관계
- 암호화폐에 대한 간략한 소개
2. 암호화폐의 역사
- 비트코인의 출현: 블록체인과 디센트럴라이제이션
- 주요 암호화폐들의 발전과 다양성 
3. 암호화폐의 경제적 영향
- 통화정책에 미치는 영향: 중앙은행과의 관계
- 국제거래에서의 암호화폐 활용
- 경제 내에서의 암호화폐로 인한 기회와 위험
4. 실제 사례 연구
- 암호화폐를 사용하는 기업들의 사례
- 국가 수준에서 암호화폐를 인정하거나 규제하는 사례
5. 미래 전망
- 암호화폐와 전통 화폐의 관계
- 디지털화 시대에서의 암호화폐의 장기적인 영향
6. 결론
- 디지털화 시대의 경제 트렌드와 암호화폐의 역할
- 글쓰기 주제와 관련된 개인적인 견해나 생각

topic 2: """축구가 세계적으로 인기있는 스포츠인 이유"""
table of contents 2:


> Finished chain.


In [8]:
print(result)

1. 서론
- 축구의 세계적인 인기와 영향력
2. 역사적 배경
- 축구의 기원과 발전
- 국제 축구 기구의 역할
3. 경기의 매력
- 스포츠의 엔터테인먼트 요소
- 다양한 팀과 선수들의 경쟁
4. 문화적 영향
- 축구와 국가 간의 연결
- 축구 문화의 형성과 전파
5. 경제적 영향
- 축구 산업의 성장과 경제적 이익
- 축구와 관련된 비즈니스 기회
6. 사회적 영향
- 축구의 사회적 통합과 다양성 증진
- 축구를 통한 사회적 문제 해결
7. 결론
- 축구의 세계적인 인기의 이유와 지속성
- 개인적인 경험과 관점
